In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:90%;}
div.CodeMirror {font-family:Consolas; font-size:18pt;}
div.output {font-size:19pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:19pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe{font-size:17pt;}
</style>
"""))

In [2]:
! pip install cx_Oracle

In [3]:
import cx_Oracle

In [4]:
dsn = cx_Oracle.makedsn(host='localhost',
                       port=1521,
                       sid='xe')
dsn

'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))'

In [5]:
conn = cx_Oracle.connect(user='scott',
                         password='tiger',
                         dsn=dsn)
conn

<cx_Oracle.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [6]:
# EMP 테이블의 모든 정보 출력
cursor = conn.cursor() # SQL문 전송, 결과 받는 객체
sql = "SELECT * FROM EMP"
cursor.execute(sql)
emps = cursor.fetchall() # sql문 결과를 튜플 list로
print(emps)

[(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20), (7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30), (7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30), (7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20), (7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30), (7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30), (7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10), (7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20), (7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10), (7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30), (7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20), (7900, 

In [7]:
import pandas as pd
emp_df = pd.DataFrame(emps)
emp_df

,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [8]:
cursor.description

[('EMPNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 0),
 ('ENAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 10, 10, None, None, 1),
 ('JOB', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 9, 9, None, None, 1),
 ('MGR', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 1),
 ('HIREDATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 1),
 ('SAL', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('COMM', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('DEPTNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 3, None, 2, 0, 1)]

In [9]:
emp_df.columns = [des[0] for des in cursor.description]
emp_df.columns

Index(['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO'], dtype='object')

In [ ]:
cursor.close()

In [14]:
# 해당 이름의 사원 출력(딕셔너리 형태): parameter 사용
    # qmark 방식: SELECT * FROM EMP WHERE ENAME = ? (DB에 따라 적용 불가)
    # named 방식(추천): SELECT * FROM EMP WHERE ENAME = :ENAME

# cursor 객체 생성
cursor = conn.cursor()

sql = "SELECT * FROM EMP WHERE ENAME = UPPER(:ENAME)"
ename = input("검색할 사원의 이름은? ")

# sql문 전송
cursor.execute(sql, {'ENAME':ename})

# 검색 결과 받기
# cursor.fetchall(): 튜플 list, 결과가 한 행도 없으면 빈 리스트
# cursor.fetchone(): 결과를 한 행씩 튜플 list (for 이용)
# cursor.fetchmany(n): 결과를 n행 튜플 list
emps = cursor.fetchall()

# description에서 key(컬럼명)들만 가져오기
keys = [description[0] for description in cursor.description]
print(emps)
print(keys)

cursor.close()

검색할 사원의 이름은? scott
[(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)]
['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']


In [20]:
# 검색한 사원정보를 저장할 딕셔너리
result = {}
if emps:
    # emps[0] 데이터를 result에 딕셔너리 형태로 저장
    emp = emps[0]
    # print(emp)
    # print(keys)
    for key, data in zip(keys, emp):
        result[key] = data
else:
    result = {'result':'not found'}

result['HIREDATE'] = result['HIREDATE'].strftime('%Y-%m-%d')
print(result)

{'EMPNO': 7788, 'ENAME': 'SCOTT', 'JOB': 'ANALYST', 'MGR': 7566, 'HIREDATE': '1982-12-09', 'SAL': 3000.0, 'COMM': None, 'DEPTNO': 20}
